# MNIST Image Classification with TensorFlow on Vertex AI

This notebook demonstrates how to implement different image models on MNIST using the [tf.keras API](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras).

## Learning objectives
1. Understand how to build a Dense Neural Network (DNN) for image classification
2. Understand how to use dropout (DNN) for image classification
3. Understand how to use Convolutional Neural Networks (CNN)
4. Know how to deploy and use an image classification model using Google Cloud's [Vertex AI](https://cloud.google.com/vertex-ai/)

First things first. Configure the parameters below to match your own Google Cloud project details.

In [ ]:
# !sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [1]:
# Here we'll show the currently installed version of TensorFlow
import tensorflow as tf
print(tf.__version__)

2.6.5


In [2]:
from datetime import datetime
import os

PROJECT = "qwiklabs-gcp-00-5761f3e728e5"  # REPLACE WITH YOUR PROJECT ID
BUCKET = "qwiklabs-gcp-00-5761f3e728e5"  # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1"  # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "dnn"  # "linear", "cnn", "dnn_dropout", or "dnn"

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "2.6"  # Tensorflow  version
os.environ["IMAGE_URI"] = os.path.join("gcr.io", PROJECT, "mnist_models")

## Building a dynamic model

In the previous notebook, <a href="1_mnist_linear.ipynb">1_mnist_linear.ipynb</a>, we ran our code directly from the notebook. In order to run it on the AI Platform, it needs to be packaged as a python module.

The boilerplate structure for this module has already been set up in the folder `mnist_models`. The module lives in the sub-folder, `trainer`, and is designated as a python package with the empty `__init__.py` (`mnist_models/trainer/__init__.py`) file. It still needs the model and a trainer to run it, so let's make them.

Let's start with the trainer file first. This file parses command line arguments to feed into the model.

In [3]:
%%writefile mnist_models/trainer/task.py
import argparse
import json
import os
import sys

from . import model


def _parse_arguments(argv):
    """Parses command-line arguments."""
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--model_type',
        help='Which model type to use',
        type=str, default='linear')
    parser.add_argument(
        '--epochs',
        help='The number of epochs to train',
        type=int, default=10)
    parser.add_argument(
        '--steps_per_epoch',
        help='The number of steps per epoch to train',
        type=int, default=100)
    parser.add_argument(
        '--job-dir',
        help='Directory where to save the given model',
        type=str, default='mnist_models/')
    return parser.parse_known_args(argv)


def main():
    """Parses command line arguments and kicks off model training."""
    args = _parse_arguments(sys.argv[1:])[0]

    # Configure path for hyperparameter tuning.
    trial_id = json.loads(
        os.environ.get('TF_CONFIG', '{}')).get('task', {}).get('trial', '')
    output_path = args.job_dir if not trial_id else args.job_dir + '/'

    model_layers = model.get_layers(args.model_type)
    image_model = model.build_model(model_layers, args.job_dir)
    model_history = model.train_and_evaluate(
        image_model, args.epochs, args.steps_per_epoch, args.job_dir)


if __name__ == '__main__':
    main()


Writing mnist_models/trainer/task.py


Next, let's group non-model functions into a util file to keep the model file simple. We'll copy over the `scale` and `load_dataset` functions from the previous lab.

In [4]:
%%writefile mnist_models/trainer/util.py
import tensorflow as tf


def scale(image, label):
    """Scales images from a 0-255 int range to a 0-1 float range"""
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.expand_dims(image, -1)
    return image, label


def load_dataset(
        data, training=True, buffer_size=5000, batch_size=100, nclasses=10):
    """Loads MNIST dataset into a tf.data.Dataset"""
    (x_train, y_train), (x_test, y_test) = data
    x = x_train if training else x_test
    y = y_train if training else y_test
    # One-hot encode the classes
    y = tf.keras.utils.to_categorical(y, nclasses)
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(scale).batch(batch_size)
    if training:
        dataset = dataset.shuffle(buffer_size).repeat()
    return dataset


Writing mnist_models/trainer/util.py


Finally, let's code the models! The [tf.keras API](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras) accepts an array of [layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers) into a [model object](https://www.tensorflow.org/api_docs/python/tf/keras/Model), so we can create a dictionary of layers based on the different model types we want to use. The below file has two functions: `get_layers` and `create_and_train_model`. We will build the structure of our model in `get_layers`. Last but not least, we'll copy over the training code from the previous lab into `train_and_evaluate`.

**TODO 1**: Define the Keras layers for a DNN model   
**TODO 2**: Define the Keras layers for a dropout model  
**TODO 3**: Define the Keras layers for a CNN model  

Hint: These models progressively build on each other. Look at the imported `tensorflow.keras.layers` modules and the default values for the variables defined in `get_layers` for guidance.

In [5]:
%%writefile mnist_models/trainer/model.py
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import (
    Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Softmax)

from . import util


# Image Variables
WIDTH = 28
HEIGHT = 28


def get_layers(
        model_type,
        nclasses=10,
        hidden_layer_1_neurons=400,
        hidden_layer_2_neurons=100,
        dropout_rate=0.25,
        num_filters_1=64,
        kernel_size_1=3,
        pooling_size_1=2,
        num_filters_2=32,
        kernel_size_2=3,
        pooling_size_2=2):
    """Constructs layers for a keras model based on a dict of model types."""
    model_layers = {
        'linear': [
            Flatten(),
            Dense(nclasses),
            Softmax()
        ],
        'dnn': [
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dense(nclasses),
            Softmax()
        ],
        'dnn_dropout': [
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dropout(dropout_rate),
            Dense(nclasses),
            Softmax()
        ],
        'cnn': [
            Conv2D(num_filters_1, kernel_size=kernel_size_1,
                   activation='relu', input_shape=(WIDTH, HEIGHT, 1)),
            MaxPooling2D(pooling_size_1),
            Conv2D(num_filters_2, kernel_size=kernel_size_2,
                   activation='relu'),
            MaxPooling2D(pooling_size_2),
            Flatten(),
            Dense(hidden_layer_1_neurons, activation='relu'),
            Dense(hidden_layer_2_neurons, activation='relu'),
            Dropout(dropout_rate),
            Dense(nclasses),
            Softmax()
        ]
    }
    return model_layers[model_type]


def build_model(layers, output_dir):
    """Compiles keras model for image classification."""
    model = Sequential(layers)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


def train_and_evaluate(model, num_epochs, steps_per_epoch, output_dir):
    """Compiles keras model and loads data into it for training."""
    mnist = tf.keras.datasets.mnist.load_data()
    train_data = util.load_dataset(mnist)
    validation_data = util.load_dataset(mnist, training=False)

    callbacks = []
    if output_dir:
        tensorboard_callback = TensorBoard(log_dir=output_dir)
        callbacks = [tensorboard_callback]

    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=num_epochs,
        steps_per_epoch=steps_per_epoch,
        verbose=2,
        callbacks=callbacks)

    if output_dir:
        export_path = os.path.join(output_dir, 'keras_export')
        model.save(export_path, save_format='tf')

    return history


Writing mnist_models/trainer/model.py


## Local Training

With everything set up, let's run locally to test the code. Some of the previous tests have been copied over into a testing script `mnist_models/trainer/test.py` to make sure the model still passes our previous checks. On `line 13`, you can specify which model types you would like to check. `line 14` and `line 15` has the number of epochs and steps per epoch respectively.

Moment of truth! Run the code below to check your models against the unit tests. If you see "OK" at the end when it's finished running, congrats! You've passed the tests!

In [6]:
!python3 -m mnist_models.trainer.test

11501568/11490434 [==============================] - 0s 0us/step
2022-10-22 20:50:54.736767: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-10-22 20:50:54.817803: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
..
*** Building model for linear ***

Epoch 1/10
100/100 - 6s - loss: 1.2843 - accuracy: 0.6770 - val_loss: 0.7742 - val_accuracy: 0.8295
Epoch 2/10
100/100 - 2s - loss: 0.6603 - accuracy: 0.8471 - val_loss: 0.5489 - val_accuracy: 0.8750
Epoch 3/10
100/100 - 1s - loss: 0.5299 - accuracy: 0.8745 - val_loss: 0.4632 - val_accuracy: 0.8873
Epoch 4/10
100/100 - 2s - loss: 0.4748 - accuracy: 0.8769 - val_loss: 0.4213 - val_accuracy: 0.8939
Epoch 5/10
100/100 - 2s - loss: 0.4236 - accuracy: 0.8916 - val_loss: 0.3890 - val_accuracy: 0.9001
Epoch 6/10
100/100 - 2s - loss: 0.4150

Now that we know that our models are working as expected, let's run it on the [Google Cloud AI Platform](https://cloud.google.com/ml-engine/docs/). We can run it as a python module locally first using the command line.

The below cell transfers some of our variables to the command line as well as create a job directory including a timestamp.

In [7]:
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
model_type = 'dnn' # "linear", "cnn", "dnn_dropout", or "dnn"

os.environ["MODEL_TYPE"] = model_type
os.environ["JOB_DIR"] = "mnist_models/models/{}_{}/".format(
    model_type, current_time)

The cell below runs the local version of the code. The epochs and steps_per_epoch flag can be changed to run for longer or shorter, as defined in our `mnist_models/trainer/task.py` file.

In [8]:
%%bash
python3 -m mnist_models.trainer.task \
    --job-dir=$JOB_DIR \
    --epochs=5 \
    --steps_per_epoch=50 \
    --model_type=$MODEL_TYPE

Epoch 1/5
50/50 - 7s - loss: 0.7803 - accuracy: 0.7894 - val_loss: 0.3843 - val_accuracy: 0.8865
Epoch 2/5
50/50 - 1s - loss: 0.3283 - accuracy: 0.9034 - val_loss: 0.2844 - val_accuracy: 0.9181
Epoch 3/5
50/50 - 1s - loss: 0.2939 - accuracy: 0.9148 - val_loss: 0.2234 - val_accuracy: 0.9353
Epoch 4/5
50/50 - 2s - loss: 0.2211 - accuracy: 0.9348 - val_loss: 0.2082 - val_accuracy: 0.9371
Epoch 5/5
50/50 - 1s - loss: 0.2190 - accuracy: 0.9336 - val_loss: 0.1733 - val_accuracy: 0.9493


2022-10-22 20:53:46.987901: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-10-22 20:53:47.418529: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-22 20:53:47.418595: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-10-22 20:53:47.418659: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-22 20:53:48.103514: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-22 20:53:52.645861: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-22 20:53:52.648674: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-10-22 20:53:52.669400: I tensorflow/core/profiler/lib/profiler_session.cc:

## Training on the cloud

Since we're using an unreleased version of TensorFlow on AI Platform, we can instead use a [Deep Learning Container](https://cloud.google.com/ai-platform/deep-learning-containers/docs/overview) in order to take advantage of libraries and applications not normally packaged with AI Platform. Below is a simple [Dockerlife](https://docs.docker.com/engine/reference/builder/) which copies our code to be used in a TF2 environment.

In [9]:
%%writefile mnist_models/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-cpu
COPY mnist_models/trainer /mnist_models/trainer
ENTRYPOINT ["python3", "-m", "mnist_models.trainer.task"]

Writing mnist_models/Dockerfile


The below command builds the image and ships it off to Google Cloud so it can be used for AI Platform. When built, it will show up [here](http://console.cloud.google.com/gcr) with the name `mnist_models`. ([Click here](https://console.cloud.google.com/cloud-build) to enable Cloud Build)

In [10]:
!docker build -f mnist_models/Dockerfile -t $IMAGE_URI ./

Sending build context to Docker daemon  4.696MB
Step 1/3 : FROM gcr.io/deeplearning-platform-release/tf2-cpu
latest: Pulling from deeplearning-platform-release/tf2-cpu

3276a519: Pulling fs layer 
546915e5: Pulling fs layer 
e62dea9f: Pulling fs layer 
b700ef54: Pulling fs layer 
e871c149: Pulling fs layer 
50e5198e: Pulling fs layer 
e6225e46: Pulling fs layer 
e871c149: Waiting fs layer 
9f65b1ef: Pulling fs layer 
50e5198e: Waiting fs layer 
9471d36c: Pulling fs layer 
2fa12294: Pulling fs layer 
9283b45a: Pulling fs layer 
09243a2a: Pulling fs layer 
700ef54: Waiting fs layer 
6225e46: Waiting fs layer 
fbc64da4: Pulling fs layer 
9471d36c: Waiting fs layer 
503b5b68: Pulling fs layer 
6c19a3f9: Pulling fs layer 
b0d99610: Pulling fs layer 
2fbc5247: Pulling fs layer 
9199c4d8: Pulling fs layer 
Digest: sha256:4982da5765431833fc94075be26dc9167091896a8107da0420a0ba20cb9e8ed0
Status: Downloaded newer image for gcr.io/deeplearning-platform-release/tf2-cpu:latest
 ---> ca2649952cdf
Ste

In [11]:
!docker push $IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/qwiklabs-gcp-00-5761f3e728e5/mnist_models]

5418f0ce: Preparing 
2a6443ca: Preparing 
b4fc3015: Preparing 
c5162b11: Preparing 
9d1239cd: Preparing 
62ed2ffb: Waiting g 
7c522b7d: Preparing 
7c6e108e: Preparing 
0fe60cdc: Preparing 
eba7f5a3: Preparing 
9ad9cb4d: Preparing 
d997fdeb: Preparing 
7c9cb170: Preparing 
3985bc23: Preparing 
31a54011: Preparing 
36c1940e: Preparing 
4e5c22e4: Preparing 
ca552b47: Preparing 
207a04f7: Preparing 
bf18a086: Preparing 
09a492af: Preparing 
bf18a086: Preparing 
834cbd4e: Preparing 
9ac077d9: Preparing 
8156a203: Layer already exists ning-platform-release/tf2-cpu latest: digest: sha256:0ef8eed5a97f15018d5554952836728ac0200a3ab77f7411a818b1b42f8f4a0c size: 5760


Finally, we can kickoff the [AI Platform training job](https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training). We can pass in our docker image using the `master-image-uri` flag.

In [17]:
current_time = datetime.now().strftime("%y%m%d_%H%M%S")
model_type = 'dnn' # "linear", "cnn", "dnn_dropout", or "dnn"

os.environ["MODEL_TYPE"] = model_type
os.environ["JOB_DIR"] = "gs://{}/mnist_{}_{}/".format(
    BUCKET, model_type, current_time)
os.environ["JOB_NAME"] = "mnist_{}_{}".format(
    model_type, current_time)

AI platform job could take around 10 minutes to complete. Enable the **AI Platform Training & Prediction API**, if required.

In [18]:
%%bash
echo $JOB_DIR $REGION $JOB_NAME
gcloud ai-platform jobs submit training $JOB_NAME \
    --staging-bucket=gs://$BUCKET \
    --region=$REGION \
    --master-image-uri=$IMAGE_URI \
    --scale-tier=BASIC_GPU \
    --job-dir=$JOB_DIR \
    -- \
    --model_type=$MODEL_TYPE

gs://qwiklabs-gcp-00-5761f3e728e5/mnist_dnn_221022_210059/ us-central1 mnist_dnn_221022_210059
jobId: mnist_dnn_221022_210059
state: QUEUED


Job [mnist_dnn_221022_210059] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe mnist_dnn_221022_210059

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs mnist_dnn_221022_210059


## Deploying and predicting with model

Once you have a model you're proud of, let's deploy it! All we need to do is give AI Platform the location of the model. Below uses the keras export path of the previous job, but `${JOB_DIR}keras_export/` can always be changed to a different path.

Uncomment the delete commands below if you are getting an "already exists error" and want to deploy a new model.

In [19]:
%%bash
gcloud config set ai_platform/region global


Updated property [ai_platform/region].


In [20]:
%%bash
MODEL_NAME="mnist"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=${JOB_DIR}keras_export/
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#yes | gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model ${MODEL_NAME} \
    --origin ${MODEL_LOCATION} \
    --framework tensorflow \
    --runtime-version=2.6

Deleting and deploying mnist dnn from gs://qwiklabs-gcp-00-5761f3e728e5/mnist_dnn_221022_210059/keras_export/ ... this will take a few minutes


Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/qwiklabs-gcp-00-5761f3e728e5/models/mnist].
Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) FAILED_PRECONDITION: Field: version.deployment_uri Error: The provided URI for model files doesn't contain any objects.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The provided URI for model files doesn't contain any objects.
    field: version.deployment_uri


CalledProcessError: Command 'b'MODEL_NAME="mnist"\nMODEL_VERSION=${MODEL_TYPE}\nMODEL_LOCATION=${JOB_DIR}keras_export/\necho "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"\n#yes | gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}\n#yes | gcloud ai-platform models delete ${MODEL_NAME}\ngcloud ai-platform models create ${MODEL_NAME} --regions $REGION\ngcloud ai-platform versions create ${MODEL_VERSION} \\\n    --model ${MODEL_NAME} \\\n    --origin ${MODEL_LOCATION} \\\n    --framework tensorflow \\\n    --runtime-version=2.6\n'' returned non-zero exit status 1.

To predict with the model, let's take one of the example images.

**TODO 4**: Write a `.json` file with image data to send to an AI Platform deployed model

In [ ]:
import json, codecs
import tensorflow as tf
import matplotlib.pyplot as plt
from mnist_models.trainer import util

HEIGHT = 28
WIDTH = 28
IMGNO = 12

mnist = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist
test_image = x_test[IMGNO]

jsondata = test_image.reshape(HEIGHT, WIDTH, 1).tolist()
json.dump(jsondata, codecs.open("test.json", "w", encoding = "utf-8"))
plt.imshow(test_image.reshape(HEIGHT, WIDTH));

Finally, we can send it to the prediction service. The output will have a 1 in the index of the corresponding digit it is predicting. Congrats! You've completed the lab!

In [ ]:
%%bash
gcloud ai-platform predict \
    --model=mnist \
    --version=${MODEL_TYPE} \
    --json-instances=./test.json

Copyright 2022 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.